In [1]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
import pickle
import numpy as np
from nltk import word_tokenize, FreqDist
import string
from os import path
from PIL import Image
import matplotlib.pyplot as plt
# from wordcloud import WordCloud, STOPWORDS,ImageColorGenerator
# from nltk.sentiment.sentiment_analyzer import SentimentAnalyzer as sa
# from nltk.classify.scikitlearn import SklearnClassifier
# from nltk.classify import ClassifierI
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import spacy
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise.model_selection import train_test_split, cross_validate
from surprise import Dataset, Reader, accuracy, SVD, NMF
reader= Reader(rating_scale=(1, 5.0))
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import Trainset
import requests

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

with open('userratingsDF.pickle', 'rb') as f:
    dfr = pickle.load(f)
with open('CategoricalItemDF.pickle', 'rb') as f:
    full = pickle.load(f)
with open('_SENT&RATINGDF.pickle', 'rb') as f:
    df1 = pickle.load(f)
with open('recdf.pickle', 'rb') as f:
    rec = pickle.load(f)
with open('ratingsmeandf.pickle', 'rb') as f:
    ratemean = pickle.load(f)

# Beginning Recommender System Using Just UserId, ItemID, and Rating

#### Item Based Collaborative Filtering probably better than User Based bc we have more users than items

In [2]:
toadd=full[['url','prodName','age','darkcircles','dry','redness','sensitive', 'oily', 'normal', 'cleanser',
       'exfoliator', 'makeup-removers', 'toner', 'mist', 'treatment', 'serum',
       'lotion', 'moisturizer', 'balm', 'oil', 'mask', 'peel', 'lip', 'eye',
       'supplement', 'tool']]

In [3]:
rec=rec.merge(toadd,on='url').drop_duplicates()
rec=rec.merge(ratemean,on='url').drop_duplicates()
rec.rename(columns={'rating_x':'rating','prodName_x':'brandName','prodName_y':'prodName'},inplace=True)
rec.drop(columns='rating_y',inplace=True)
rec.head()

,brandName,rating,url,user,prodName,age,darkcircles,dry,redness,sensitive,...,moisturizer,balm,oil,mask,peel,lip,eye,supplement,tool,rating_count
0,Paula's Choice,5,11174178,becky c,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
1,Paula's Choice,5,11174178,jess,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
2,Paula's Choice,5,11174178,lisa,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
3,Paula's Choice,5,11174178,stella2,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60
4,Paula's Choice,5,11174178,carli,Paula's Choice Skin Perfecting 2% BHA Liquid E...,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,60


In [4]:
data = Dataset.load_from_df(rec[['user','url','rating']],reader)

In [5]:
dataset = data.build_full_trainset()
print('Number of users: ',dataset.n_users,'\n')
print('Number of items: ',dataset.n_items)

Number of users:  1378 

Number of items:  514


## Trying to Minimize RMSE

#### SVD Grid Search

In [6]:
params = {'n_factors' :[10,20,30,40,50,60,70,80,90,100,200],
         'reg_all':[0.02,0.05,0.1,0.4,0.5,0.6],'n_epochs': [5, 10],
         'lr_all': [0.002, 0.005],'reg_all': [0.4, 0.6]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)

In [7]:
print(g_s_svd.best_score['rmse'])
print(g_s_svd.best_params['rmse'])

0.9633888298748428
{'n_factors': 40, 'reg_all': 0.4, 'n_epochs': 10, 'lr_all': 0.005}


#### NMF Grid Search

In [8]:
params = {'n_factors' :[5,10,20,30,40,50,60,70,80,90,100,200],
          'n_epochs': [5, 10]}
g_s_nmf = GridSearchCV(NMF,param_grid=params,n_jobs=-1)
g_s_nmf.fit(data)

In [9]:
print(g_s_nmf.best_score['rmse'])
print(g_s_nmf.best_params['rmse'])

0.9922151538829374
{'n_factors': 200, 'n_epochs': 10}


## KNN

### SIM_OPTIONS: Cosine Item Based

#### KNN Basic

In [160]:
knn_basic = KNNBasic(sim_options={'name':'cosine','user_based':False})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([1.00668756, 0.99505168, 0.90076885, 1.08352872, 0.89960236]))
('test_mae', array([0.69376143, 0.69034835, 0.64098749, 0.72584173, 0.63419717]))
('fit_time', (0.005627870559692383, 0.005323886871337891, 0.005083799362182617, 0.005198955535888672, 0.004731893539428711))
('test_time', (0.0017137527465820312, 0.0016748905181884766, 0.0016651153564453125, 0.0017728805541992188, 0.0018031597137451172))
-----------------------
0.9771278351356145


#### KNN Baseline

In [163]:
knn_baseline = KNNBaseline(sim_options={'name':'cosine','user_based':False})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.99657255, 0.95340813, 1.0189458 , 0.98019162, 0.9073236 ]))
('test_mae', array([0.67919473, 0.67039211, 0.69639837, 0.67608548, 0.63110021]))
('fit_time', (0.012480974197387695, 0.012412786483764648, 0.009212017059326172, 0.010272026062011719, 0.010171175003051758))
('test_time', (0.002516031265258789, 0.0021598339080810547, 0.001950979232788086, 0.0018680095672607422, 0.0019690990447998047))
-----------------------
0.9712883383716051


#### KNN With Means

In [178]:
knn_means = KNNWithMeans(sim_options={'name':'cosine','user_based':False})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.96048315, 1.05057543, 1.04637594, 0.92844599, 0.99753315]))
('test_mae', array([0.65257879, 0.69738957, 0.68658064, 0.63740162, 0.67350202]))
('fit_time', (0.012176036834716797, 0.012534856796264648, 0.01201009750366211, 0.010139226913452148, 0.008389949798583984))
('test_time', (0.0024878978729248047, 0.0023200511932373047, 0.002270936965942383, 0.002065896987915039, 0.002001047134399414))
-----------------------
0.9966827339456261


### SIM_OPTIONS: Cosine USer Based

#### KNN Basic

In [164]:
knn_basic = KNNBasic(sim_options={'name':'cosine','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([1.02947006, 0.97202445, 0.96806476, 0.94829387, 0.97347879]))
('test_mae', array([0.70727482, 0.66659346, 0.67929112, 0.6617394 , 0.67282127]))
('fit_time', (0.03415393829345703, 0.034859657287597656, 0.03474569320678711, 0.032071828842163086, 0.03397202491760254))
('test_time', (0.0023200511932373047, 0.0019750595092773438, 0.0021491050720214844, 0.0019021034240722656, 0.0018661022186279297))
-----------------------
0.9782663863348275


#### KNN Baseline

In [165]:
knn_baseline = KNNBaseline(sim_options={'name':'cosine','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.90616494, 1.10353407, 0.85771825, 0.99883143, 0.98322041]))
('test_mae', array([0.64120325, 0.72741781, 0.61913925, 0.6982636 , 0.67150947]))
('fit_time', (0.050257205963134766, 0.0318598747253418, 0.02761101722717285, 0.029393911361694336, 0.04259991645812988))
('test_time', (0.0021359920501708984, 0.0020210742950439453, 0.0019791126251220703, 0.002002239227294922, 0.0018858909606933594))
-----------------------
0.9698938215232413


#### KNN With Means

In [179]:
knn_means = KNNWithMeans(sim_options={'name':'cosine','user_based':True})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.17846214, 0.95436375, 0.99328449, 1.02427175, 0.92085753]))
('test_mae', array([0.74239993, 0.65674161, 0.66069284, 0.69144495, 0.60341413]))
('fit_time', (0.04790687561035156, 0.03203082084655762, 0.032475948333740234, 0.03106093406677246, 0.03368401527404785))
('test_time', (0.0022640228271484375, 0.00199127197265625, 0.002045869827270508, 0.0019681453704833984, 0.0020809173583984375))
-----------------------
1.0142479335806986


### SIM_OPTIONS Peasrson Item Based

#### KNN Basic

In [166]:
knn_basic = KNNBasic(sim_options={'name':'pearson','user_based':False})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([1.04111251, 1.03871506, 0.96678572, 1.00103062, 0.83363746]))
('test_mae', array([0.70244541, 0.70764459, 0.66601976, 0.6868882 , 0.62915486]))
('fit_time', (0.008600711822509766, 0.008198022842407227, 0.007836341857910156, 0.00902700424194336, 0.0051670074462890625))
('test_time', (0.0020520687103271484, 0.0019009113311767578, 0.0018107891082763672, 0.0019309520721435547, 0.0017290115356445312))
-----------------------
0.976256272730178


#### KNN Baseline

In [167]:
knn_baseline = KNNBaseline(sim_options={'name':'pearson','user_based':False})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.953247  , 0.90237868, 1.04123358, 0.98036736, 0.93852743]))
('test_mae', array([0.66678094, 0.64986059, 0.7036908 , 0.66638486, 0.6517893 ]))
('fit_time', (0.014440774917602539, 0.013377904891967773, 0.01069498062133789, 0.011998891830444336, 0.008732795715332031))
('test_time', (0.002279043197631836, 0.0019481182098388672, 0.0019159317016601562, 0.0018358230590820312, 0.0019192695617675781))
-----------------------
0.963150808567257

#### KNN With Means

In [180]:
knn_means = KNNWithMeans(sim_options={'name':'pearson','user_based':False})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.88727882, 0.97497212, 0.94923614, 1.06099029, 1.12059445]))
('test_mae', array([0.62925003, 0.67164012, 0.63346716, 0.69347224, 0.71916346]))
('fit_time', (0.016582012176513672, 0.014825105667114258, 0.010214090347290039, 0.009563922882080078, 0.010123014450073242))
('test_time', (0.0025670528411865234, 0.002286195755004883, 0.002138853073120117, 0.002012968063354492, 0.0020041465759277344))
-----------------------
0.9986143649873206


### SIM_OPTIONS Pearson User Based

#### KNN Basic

In [168]:
knn_basic = KNNBasic(sim_options={'name':'pearson','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.89994467, 1.03028195, 0.98292124, 0.93622735, 1.03921655]))
('test_mae', array([0.65185198, 0.69744792, 0.68166212, 0.66203521, 0.70006992]))
('fit_time', (0.047106027603149414, 0.03520488739013672, 0.03540992736816406, 0.049690961837768555, 0.04812908172607422))
('test_time', (0.0020258426666259766, 0.002270936965942383, 0.002148151397705078, 0.0022161006927490234, 0.0020170211791992188))
-----------------------
0.9777183528170559


#### KNN Basline

In [169]:
knn_baseline = KNNBaseline(sim_options={'name':'pearson','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.00966243, 0.96518488, 0.90872037, 0.97788387, 0.96350045]))
('test_mae', array([0.68850616, 0.69046209, 0.63864346, 0.66575563, 0.66410403]))
('fit_time', (0.06789612770080566, 0.03771686553955078, 0.054840087890625, 0.033451080322265625, 0.031995296478271484))
('test_time', (0.0022788047790527344, 0.0021610260009765625, 0.002151966094970703, 0.0021021366119384766, 0.0020418167114257812))
-----------------------
0.9649903990713978


#### KNN With Means

In [181]:
knn_means = KNNWithMeans(sim_options={'name':'pearson','user_based':True})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.0572003 , 1.11626738, 0.98923093, 0.98275035, 0.9225197 ]))
('test_mae', array([0.70804349, 0.71925655, 0.65750624, 0.63128683, 0.63137946]))
('fit_time', (0.04231595993041992, 0.03513479232788086, 0.04136991500854492, 0.04080772399902344, 0.03638315200805664))
('test_time', (0.002267122268676758, 0.001992940902709961, 0.002068042755126953, 0.0020580291748046875, 0.0020570755004882812))
-----------------------
1.0135937303324263


### SIM_OPTIONS MSD Item Based

#### KNN Basic

In [170]:
knn_basic = KNNBasic(sim_options={'name':'msd','user_based':False})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([1.03411959, 0.96936912, 0.94287448, 1.01224547, 0.9321553 ]))
('test_mae', array([0.69995339, 0.67132203, 0.66767139, 0.68783391, 0.65565277]))
('fit_time', (0.0053670406341552734, 0.004289150238037109, 0.004187822341918945, 0.004263162612915039, 0.0028450489044189453))
('test_time', (0.0020008087158203125, 0.001889944076538086, 0.0017838478088378906, 0.0017600059509277344, 0.0020117759704589844))
-----------------------
0.9781527927730167


#### KNN Baseline

In [171]:
knn_baseline = KNNBaseline(sim_options={'name':'msd','user_based':False})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.06071802, 0.91976907, 1.00552527, 0.8073787 , 1.0119366 ]))
('test_mae', array([0.70436286, 0.63781604, 0.69272798, 0.60127685, 0.69121273]))
('fit_time', (0.009452104568481445, 0.009010791778564453, 0.008800029754638672, 0.0069561004638671875, 0.007444858551025391))
('test_time', (0.0025141239166259766, 0.002216815948486328, 0.0019638538360595703, 0.0018341541290283203, 0.0018551349639892578))
-----------------------
0.961065532895623


#### KNN With Means

In [182]:
knn_means = KNNWithMeans(sim_options={'name':'msd','user_based':False})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.12188373, 0.86931771, 0.96506838, 0.99536495, 1.02359539]))
('test_mae', array([0.7459268 , 0.60625738, 0.66040044, 0.66526108, 0.66939955]))
('fit_time', (0.008738994598388672, 0.009012937545776367, 0.008905887603759766, 0.010341167449951172, 0.00745391845703125))
('test_time', (0.002782106399536133, 0.002331256866455078, 0.002315044403076172, 0.0022940635681152344, 0.0019571781158447266))
-----------------------
0.9950460327830474


### SIM_OPTIONS MSD USER BASED

#### KNN Basic

In [172]:
knn_basic = KNNBasic(sim_options={'name':'msd','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.98720192, 1.0063807 , 1.00265125, 1.00515674, 0.89843609]))
('test_mae', array([0.68556151, 0.69933486, 0.69381073, 0.67917458, 0.6322811 ]))
('fit_time', (0.02790522575378418, 0.02241992950439453, 0.020099163055419922, 0.018820762634277344, 0.018208980560302734))
('test_time', (0.0022149085998535156, 0.003343820571899414, 0.0018491744995117188, 0.0021162033081054688, 0.0018188953399658203))
-----------------------
0.9799653392449839


#### KNN Baseline

In [173]:
knn_baseline = KNNBaseline(sim_options={'name':'msd','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.91782599, 0.96559899, 1.10288513, 0.8757128 , 0.96845038]))
('test_mae', array([0.66141492, 0.67388105, 0.73202743, 0.62093307, 0.65772308]))
('fit_time', (0.03564596176147461, 0.03364706039428711, 0.02142810821533203, 0.03197622299194336, 0.020447254180908203))
('test_time', (0.0024008750915527344, 0.0019941329956054688, 0.0020079612731933594, 0.0021347999572753906, 0.002004384994506836))
-----------------------
0.9660946560028156


#### KNN With Means

In [183]:
knn_means = KNNWithMeans(sim_options={'name':'msd','user_based':True})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.99554736, 0.99535769, 1.10797529, 0.99434918, 1.05355276]))
('test_mae', array([0.67310369, 0.6685566 , 0.70870654, 0.65106703, 0.70899264]))
('fit_time', (0.030163288116455078, 0.026775836944580078, 0.02669978141784668, 0.027655839920043945, 0.027059078216552734))
('test_time', (0.002402782440185547, 0.002025127410888672, 0.0020821094512939453, 0.0020999908447265625, 0.0020627975463867188))
-----------------------
1.0293564553251282


### SIM_OPTIONS Pearson Baseline Item Based

#### KNN Basic

In [174]:
knn_basic = KNNBasic(sim_options={'name':'pearson_baseline','user_based':False})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.9969231 , 0.92427028, 1.05836774, 1.04629287, 0.85834334]))
('test_mae', array([0.68457098, 0.65852128, 0.70811521, 0.71824465, 0.62407221]))
('fit_time', (0.010994911193847656, 0.009063243865966797, 0.008513927459716797, 0.009830951690673828, 0.008569717407226562))
('test_time', (0.0016722679138183594, 0.0016198158264160156, 0.0014390945434570312, 0.001644134521484375, 0.0014200210571289062))
-----------------------
0.9768394654093149


### ** KNN Baseline !!!! BEST RMSE

In [175]:
knn_baseline = KNNBaseline(sim_options={'name':'pearson_baseline','user_based':False})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.97926163, 0.87623109, 0.96970176, 0.96479831, 1.00857411]))
('test_mae', array([0.6777355 , 0.63316169, 0.66862343, 0.66778598, 0.6866294 ]))
('fit_time', (0.012312173843383789, 0.011269092559814453, 0.010456085205078125, 0.00926828384399414, 0.010927915573120117))
('test_time', (0.0024509429931640625, 0.0019788742065429688, 0.001992940902709961, 0.001795053482055664, 0.0018260478973388672

#### KNN With Means

In [184]:
knn_means = KNNWithMeans(sim_options={'name':'pearson_baseline','user_based':False})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
('test_rmse', array([1.03932539, 1.11346106, 0.89850938, 1.04320803, 0.95089436]))
('test_mae', array([0.69259128, 0.71826159, 0.63631641, 0.69066909, 0.65312325]))
('fit_time', (0.014931917190551758, 0.013577938079833984, 0.012480020523071289, 0.011180877685546875, 0.01148223876953125))
('test_time', (0.0020689964294433594, 0.002034902572631836, 0.001811981201171875, 0.001622915267944336, 0.001596212387084961))

### SIM_OPTIONS Pearson Baseline User Based

#### KNN Basic

In [176]:
knn_basic = KNNBasic(sim_options={'name':'pearson_baseline','user_based':True})
cv_knn_basic= cross_validate(knn_basic,data,n_jobs=-1)
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([1.11421503, 0.95657628, 1.02930919, 0.85511938, 0.92133908]))
('test_mae', array([0.73381979, 0.67953315, 0.69726864, 0.63109111, 0.65039532]))
('fit_time', (0.025043010711669922, 0.036263227462768555, 0.039808034896850586, 0.029501914978027344, 0.026002883911132812))
('test_time', (0.0021092891693115234, 0.001729726791381836, 0.0017011165618896484, 0.0016560554504394531, 0.001764059066772461))
-----------------------
0.9753117943916694


#### KNN Baseline

In [177]:
knn_baseline = KNNBaseline(sim_options={'name':'pearson_baseline','user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)
for i in cv_knn_baseline.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_baseline['test_rmse']))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.87313227, 1.00678901, 0.90559295, 0.93213092, 1.1045207 ]))
('test_mae', array([0.62885345, 0.68727673, 0.63377427, 0.65928471, 0.74042353]))
('fit_time', (0.04716897010803223, 0.03551912307739258, 0.027496814727783203, 0.02474188804626465, 0.04282188415527344))
('test_time', (0.0022509098052978516, 0.0020427703857421875, 0.0018951892852783203, 0.0019490718841552734, 0.0019609928131103516)

#### KNN With Means

In [185]:
knn_means = KNNWithMeans(sim_options={'name':'pearson_baseline','user_based':True})
cv_knn_means = cross_validate(knn_means,data)
for i in cv_knn_means.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_means['test_rmse']))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
('test_rmse', array([0.94932918, 1.07861286, 0.98616397, 1.06842821, 0.98857138]))
('test_mae', array([0.63575065, 0.70899668, 0.65507996, 0.6944046 , 0.6550695 ]))
('fit_time', (0.03914022445678711, 0.033740997314453125, 0.03273916244506836, 0.03462409973144531, 0.0338749885559082))
('test_time', (0.0020737648010253906, 0.0016651153564453125, 0.001725912094116211, 0.0017719268798828125, 0.0018591880798339844))


# Looking at predictions for best classification model

In [448]:
sim_options={'name':'pearson_baseline','user_based':False}
knn_baseline = knns.KNNBaseline(sim_options=sim_options)
knn_baseline.fit(dataset)
predictions = knn_baseline.test(test)
print(accuracy.rmse(predictions))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.2019
0.20191515211398295


In [41]:
# knn_baseline.test(test)

# Collaborative Filtering, Item Based

## Trying to Predict for New Users

In [20]:
def skin_rater(df,num, typ=None):
    rating_list = []
    movie_df=pd.DataFrame()
    raw_uid=input('type username: ')
    skintype=input('what is your skin type? (oily, dry, sensitive, normal, combination): ')
    if 'oily':
        movie_df.append(df[df['oily']==1])
    elif skintype=='sensitive':
        movie_df=df[df['sensitive']==1]
    elif skintype=='dry':
        movie_df=df[df['dry']==1]
    elif skintype=='normal' or skintype=='combination':
        movie_df=df[df['normal']==1]
    else:
        movie_df=rec
    while num > 0:
        if typ:
            p = movie_df[movie_df['types'].str.contains(genre)].sample(1)
        else:
            p = movie_df.sample(1)
        print(p[['brandName','prodName','url']])
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'user':raw_uid,'url':p['url'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return(rating_list,raw_uid)
def rank_prods_for_user(df,raw_uid):
    list_of_prods=[]
    for u in df['url'].unique():
            list_of_prods.append((u,knn_baseline.predict(raw_uid,u)[3]))
    ranked_prods = sorted(list_of_prods,key=lambda x:x[1],reverse=True)
    return (ranked_prods)
def recommended_products(user_ratings,df,n):
    for idx, re in enumerate(user_ratings):
        title = df.loc[df['url'] == int(re[0])]['prodName']
        u=re[0]
        print('Recommendation # ',idx+1,'|| ', 'url:', u, '||','product: ', rec[rec['url']==u]['prodName'].drop_duplicates(),'\n')
        n-= 1
        if n == 0:
            break

In [27]:
r,raw_uid=skin_rater(rec,3)

type username: m
what is your skin type? (oily, dry, sensitive, normal, combination): normal
        brandName                                        prodName       url
1458  Erno Laszlo  Erno Laszlo Phelityl Pre Cleansing Oil (6.8oz)  10941207
How do you rate this movie on a scale of 1-5, press n if you have not seen :
1
         brandName                        prodName       url
538  SkinCeuticals  SkinCeuticals Phloretin CF Gel  11292113
How do you rate this movie on a scale of 1-5, press n if you have not seen :
1
       brandName                                   prodName       url
1419  SkinMedica  SkinMedica Rejuvenative Moisturizer (2oz)  11289673
How do you rate this movie on a scale of 1-5, press n if you have not seen :
1


In [28]:
new_df=rec[['user','url','rating']]
new_ratings_df = new_df.append(r,ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df,reader)
new_data=new_data.build_full_trainset()
sim_options={'name':'pearson_baseline','user_based':False}
knn_baseline=knns.KNNBaseline(sim_options=sim_options)
knn_baseline.fit(new_data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [30]:
recommended_products(ranked,rec,3)

Recommendation #  1 ||  url: 11286360 || product:  1061    Avène Thermal Spring Water 10.58oz
Name: prodName, dtype: object 

Recommendation #  2 ||  url: 10375555 || product:  174    Murad Pure Skin Clarifying Dietary Supplement ...
Name: prodName, dtype: object 

Recommendation #  3 ||  url: 11524845 || product:  1140    FOREO LUNA™ mini 2 - Fuchsia
Name: prodName, dtype: object 



## Description TFIDF and Cosine Similarity !?!??!?!?!??! LOOKING AT REVIEW CONTENT AND TITLES

In [57]:
df1['description']=df1['title']+df1['content']

In [58]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df1['description'])

In [59]:
tfidf_matrix.shape

(1685, 39382)

In [60]:
full.shape

(1701, 30)

In [61]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [62]:
cosine_sim

array([[1.        , 0.03513622, 0.02979474, ..., 0.00730602, 0.        ,
        0.00399307],
       [0.03513622, 1.        , 0.02854128, ..., 0.        , 0.        ,
        0.        ],
       [0.02979474, 0.02854128, 1.        , ..., 0.        , 0.        ,
        0.00425702],
       ...,
       [0.00730602, 0.        , 0.        , ..., 1.        , 0.        ,
        0.05258715],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.00154483],
       [0.00399307, 0.        , 0.00425702, ..., 0.05258715, 0.00154483,
        1.        ]])

In [63]:
# def item(id):  
#     return df1.loc[df1['url'] == id]['description'].tolist()
# # Just reads the results out of the dictionary.def 
# def recommend(item_id, num):
#     print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")   
#     print("-------")    
#     recs = results[item_id][:num]   
#     for rec in recs: 
#         print("Recommended: " + item(rec[1]) + " (score:" +      str(rec[0]) + ")")

In [64]:
# df1.reset_index(inplace=True)
# products = df1['url']
# indices = pd.Series(df1.index, index=df1['url'])

In [65]:
# def get_recommendations(product):
#     idx = indices[products]
#     sim_scores = list(enumerate(cosine_sim[idx]))
# #     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:31]
#     prod_indices = [i[0] for i in sim_scores]
#     return products.iloc[prod_indices]

In [67]:
# get_recommendations(11429304).drop_duplicates()

In [68]:
# recommend(10362879,5)

In [69]:
alldf=dfr[['url','prodName','rating','user']]

In [70]:
alldf.groupby(['url','user','prodName']).mean().head()

rating
url      user      prodName        
10236357 adrian    Anthony        5
         george    Anthony        5
         kimber    Anthony        5
         mike      Anthony        5
         westtexas Anthony        5

In [44]:
train, test = train_test_split(data,test_size=0.2)

## Does this produce recommendations or is it messed up?

In [169]:
# def skincare_rater(df,num, typ=None, problem=None):
#     userID = 1000
#     rating_list = []
#     while num > 0:
#         if typ:
#             prod = df[df[''].str.contains(typ)].sample(1)
#         else:
#             sample=df.sample(1)
#             brand=sample[['brandName']]
#             product=sample[['prodName']]
#             url=sample[['url']]
#             prod = sample[['brandName','prodName','url']]
#         print(prod)
#         rating = input('How do you rate this product on a scale of 1-5, press n if you have not used :\n')
#         if rating == 'n':
#             continue
#         else:
#             rating_one_prod=df.sample(1)[['url','prodName']]
#             rating_one_prod['user']=1000
#             rating_one_prod['rating']=rating
#             ratedict={'url':url,'brandName':brand,'prodName':product}
#             rating_list.append(rating_one_prod) 
#             num -= 1
#     return rating_list